In [1]:
import torch

print("Torch version:", torch.__version__)

print("CUDA available:", torch.cuda.is_available())

print("CUDA version:", torch.version.cuda)

print("Number of GPUs:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


Torch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
Number of GPUs: 1
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
pip install torchtext==0.6.0


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import random
import os
import glob

# Thiết lập seed để đảm bảo tính tái lập
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_seed()

In [4]:
# Tải bộ dữ liệu TextData
data_dir = 'TextData'
classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
label_map = {class_name: idx for idx, class_name in enumerate(classes)}

print(f"Found {len(classes)} classes: {classes}")

all_data = []
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    file_paths = glob.glob(os.path.join(class_dir, '*.txt'))
    for file_path in file_paths:
        try:
            with open(file_path, 'r', encoding='utf-16') as f:
                text = f.read().strip()
                if text: # Đảm bảo văn bản không rỗng
                    all_data.append((text, label_map[class_name]))
        except Exception as e:
            print(f"Error reading {file_path}: {e}")

print(f"Total samples: {len(all_data)}")

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42, stratify=[label for _, label in all_data])

print(f"Train samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

Found 27 classes: ['Am nhac', 'Am thuc', 'Bat dong san', 'Bong da', 'Chung khoan', 'Cum ga', 'Cuoc song do day', 'Du hoc', 'Du lich', 'Duong vao WTO', 'Gia dinh', 'Giai tri tin hoc', 'Giao duc', 'Gioi tinh', 'Hackers va Virus', 'Hinh su', 'Khong gian song', 'Kinh doanh quoc te', 'Lam dep', 'Loi song', 'Mua sam', 'My thuat', 'San khau dien anh', 'San pham tin hoc moi', 'Tennis', 'The gioi tre', 'Thoi trang']
Total samples: 14375
Train samples: 11500
Test samples: 2875


In [5]:
# Tokenization và Từ vựng
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

counter = Counter()
for text, _ in train_data:
    counter.update(tokenizer(text))

# Tạo từ vựng
vocab = Vocab(counter, specials=['<unk>', '<pad>'])

# Ánh xạ các token đặc biệt sang chỉ số unk/pad
UNK_IDX = vocab['<unk>']
PAD_IDX = vocab['<pad>']

# Hàm mã hóa
MAX_LEN = 256

def encode(text):
    tokens = tokenizer(text)
    ids = [vocab[token] if token in vocab.stoi else UNK_IDX for token in tokens][:MAX_LEN]
    if len(ids) < MAX_LEN:
        ids += [PAD_IDX] * (MAX_LEN - len(ids))
    return ids

# Dataset tùy chỉnh
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text, label = self.data[idx]
        return torch.tensor(encode(text), dtype=torch.long), torch.tensor(label, dtype=torch.long)

train_dataset = TextDataset(train_data)
test_dataset = TextDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [11]:
# Mô hình Transformer Encoder
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, hidden_dim, num_layers, num_classes, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=PAD_IDX)
        self.pos_embedding = nn.Embedding(MAX_LEN, embed_dim)
        self.dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=num_heads,
            dim_feedforward=hidden_dim, 
            dropout=dropout, 
            batch_first=True,
            norm_first=True  # Chuẩn hóa trước lớp
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Attention pooling thay vì mean pooling
        self.attention = nn.Linear(embed_dim, 1)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        padding_mask = (x == PAD_IDX)  # [batch_size, seq_len]
        
        positions = torch.arange(0, x.size(1), device=x.device).unsqueeze(0)
        x = self.embedding(x) + self.pos_embedding(positions)
        x = self.dropout(x)

        # Transformer encoder với padding mask
        x = self.transformer_encoder(x, src_key_padding_mask=padding_mask)
        
        # Attention pooling
        attn_weights = torch.softmax(self.attention(x), dim=1)  # [batch_size, seq_len, 1]
        x = torch.sum(x * attn_weights, dim=1)  # [batch_size, embed_dim]
        
        return self.fc(x)

# Thiết lập mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = len(classes)
print(f"Initializing model with {num_classes} classes.")

# Cải thiện: Giảm độ phức tạp mô hình và tăng regularization
model = TransformerClassifier(
    vocab_size=len(vocab),
    embed_dim=256,
    num_heads=4,
    hidden_dim=512,
    num_layers=2,       # Giảm từ 4 xuống 2 lớp để tránh overfitting trên tập dữ liệu nhỏ/vừa
    num_classes=num_classes,
    dropout=0.3         # Tăng dropout từ 0.2 lên 0.3
).to(device)

criterion = nn.CrossEntropyLoss()
# Thêm weight_decay để phạt các trọng số lớn (L2 regularization)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

Initializing model with 27 classes.


c:\Users\nguye\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [12]:
from tqdm import tqdm

# Hàm huấn luyện
def train(model, loader):
    model.train()
    total_loss = 0
    loop = tqdm(loader, desc="Training", leave=False)
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    return total_loss / len(loader)

# Hàm đánh giá
def evaluate(model, loader):
    model.eval()
    preds, targets = [], []
    total_loss = 0
    loop = tqdm(loader, desc="Evaluating", leave=False)
    with torch.no_grad():
        for x, y in loop:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            preds.extend(pred.cpu().numpy())
            targets.extend(y.cpu().numpy())
    return total_loss / len(loader), accuracy_score(targets, preds)

# Chạy huấn luyện với early stopping
best_acc = 0
patience = 3
trigger_times = 0
train_losses = []

for epoch in range(30):
    print(f"\nEpoch {epoch+1}")
    train_loss = train(model, train_loader)
    test_loss, test_acc = evaluate(model, test_loader)
    scheduler.step(test_loss)
    
    train_losses.append(train_loss)
    print(f"Train Loss: {train_loss:.4f} - Test Loss: {test_loss:.4f} - Test Accuracy: {test_acc:.4f}")
    
    if test_acc > best_acc:
        best_acc = test_acc
        trigger_times = 0
        torch.save(model.state_dict(), "transformer_text_classification.pth")
        print(f"Saved best model to transformer_text_classification.pth")
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered!")
            break


Epoch 1


Train Loss: 1.6886 - Test Loss: 1.0260 - Test Accuracy: 0.7009
Saved best model to transformer_text_classification.pth

Epoch 2


Train Loss: 0.8448 - Test Loss: 0.7632 - Test Accuracy: 0.7805
Saved best model to transformer_text_classification.pth

Epoch 3


Train Loss: 0.6568 - Test Loss: 0.6564 - Test Accuracy: 0.8024
Saved best model to transformer_text_classification.pth

Epoch 4


Train Loss: 0.5405 - Test Loss: 0.6504 - Test Accuracy: 0.8157
Saved best model to transformer_text_classification.pth

Epoch 5


...

Train Loss: 0.3030 - Test Loss: 0.6143 - Test Accuracy: 0.8393
Saved best model to transformer_text_classification.pth

Epoch 10


Train Loss: 0.2126 - Test Loss: 0.5690 - Test Accuracy: 0.8623
Saved best model to transformer_text_classification.pth

Epoch 11


Train Loss: 0.1831 - Test Loss: 0.6059 - Test Accuracy: 0.8557

Epoch 12


Train Loss: 0.1669 - Test Loss: 0.6359 - Test Accuracy: 0.8616

Epoch 13


Train Loss: 0.1227 - Test Loss: 0.6239 - Test Accuracy: 0.8605
Early stopping triggered!


In [13]:
label_names = classes

def predict_batch(text_list):
    model.eval()
    results = []
    with torch.no_grad():
        for text in text_list:
            encoded = encode(text)
            tensor = torch.tensor(encoded, dtype=torch.long).unsqueeze(0).to(device)
            output = model(tensor)
            pred_idx = torch.argmax(output, dim=1).item()
            pred_label = label_names[pred_idx]
            results.append((text, pred_label))
    return results

In [ ]:
model.load_state_dict(torch.load("transformer_text_classification.pth", weights_only=True))
model.eval()
model.to(device)

vietnamese_samples = [
    "Đội tuyển bóng đá Việt Nam đã giành chiến thắng thuyết phục trước đối thủ.",
    "Thị trường chứng khoán hôm nay ghi nhận mức tăng điểm kỷ lục.",
    "Ca sĩ Mỹ Tâm vừa ra mắt album mới với nhiều ca khúc hit.",
    "Giá vàng trong nước tiếp tục tăng cao do ảnh hưởng của thị trường thế giới.",
    "Các nhà khoa học vừa phát hiện một loài động vật mới tại vườn quốc gia.",
    "Bộ Giáo dục và Đào tạo công bố phương án thi tốt nghiệp THPT năm nay.",
    "Du lịch Việt Nam đang thu hút ngày càng nhiều khách quốc tế.",
    "Món phở Hà Nội luôn là niềm tự hào của ẩm thực Việt Nam.",
    "Tình hình giao thông tại các thành phố lớn đang ngày càng ùn tắc."
]

predicted = predict_batch(vietnamese_samples)
for text, label in predicted:
    print(f"{text} -> [{label}]")

Đội tuyển bóng đá Việt Nam đã giành chiến thắng thuyết phục trước đối thủ. -> [Bong da]
Thị trường chứng khoán hôm nay ghi nhận mức tăng điểm kỷ lục. -> [Kinh doanh quoc te]
Ca sĩ Mỹ Tâm vừa ra mắt album mới với nhiều ca khúc hit. -> [Am nhac]
Giá vàng trong nước tiếp tục tăng cao do ảnh hưởng của thị trường thế giới. -> [Kinh doanh quoc te]
Các nhà khoa học vừa phát hiện một loài động vật mới tại vườn quốc gia. -> [Giao duc]
Bộ Giáo dục và Đào tạo công bố phương án thi tốt nghiệp THPT năm nay. -> [Giao duc]
Du lịch Việt Nam đang thu hút ngày càng nhiều khách quốc tế. -> [Du lich]
Công nghệ AI đang thay đổi cách chúng ta làm việc và sinh sống. -> [Gioi tinh]
Món phở Hà Nội luôn là niềm tự hào của ẩm thực Việt Nam. -> [Am thuc]
Tình hình giao thông tại các thành phố lớn đang ngày càng ùn tắc. -> [Kinh doanh quoc te]
